In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-133257")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-133257
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-133257


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cluster1"
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
cluster = ComputeTarget.create(ws, cluster_name, compute_config)
cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
os.listdir()

['.ipynb_checkpoints',
 'outputs',
 'train.py',
 'udacity-project (1).ipynb',
 'Users']

In [20]:
from azureml.core.environment import Environment
env=Environment.get(ws,name="southcentralus")

Exception: Error retrieving the environment definition. Code: 404
: {
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "No definitions exist for environment southcentralus",
    "messageFormat": null,
    "messageParameters": null,
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": {
      "code": "NotFoundError",
      "innerError": null
    },
    "debugInfo": null
  },
  "correlation": {
    "operation": "f225fd528d2d054da513d0067c0c4c01",
    "request": "da13b95dae38354f"
  },
  "environment": "southcentralus",
  "location": "southcentralus",
  "time": "2021-01-04T06:14:16.6897057+00:00",
  "componentName": "environment-management"
}

In [15]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
import os

# Specify parameter sampler

d = {"--c": choice(1,2), "--max_iter": choice(100,200,300)}
ps = RandomParameterSampling(d)

# Specify a Policy
policy = BanditPolicy (evaluation_interval= 2, slack_factor= 0.1)

# Create a SKLearn estimator for use with train.py
SKLearn(source_directory='./', entry_script='train.py', compute_target=cluster)
#est = ScriptRunConfig(source_directory='./', script='train.py', compute_target=cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (hyperparameter_sampling = ps, primary_metric_name = 'Accuracy', primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, policy = policy, max_total_runs=15, max_concurrent_runs = 4, estimator = est)
                                   

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [17]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)

AttributeError: 'ScriptRunConfig' object has no attribute '_compute_target'

In [ ]:
# Get your best run and save the model from that run.

import joblib
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())

model = best_run.register_model(model_name='Bank_marketing_model_hyperdrive', model_path='./')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path_url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(path = path_url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

auto_ml_run = experiment.submit(config = automl_config, show_output = True)
RunDetails(auto_ml_run).show()

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = auto_ml_run.get_output()
print(best_run)
print(fitted_model)

model_ml = best_run.register_model(model_name='Bank_marketing_model_auto_ml', model_path='./')